In [1]:
!pip3 install catboost
!pip install hyperopt

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import hyperopt
from numpy.random import RandomState
from catboost.utils import select_threshold

In [3]:
csv_path = "drive/MyDrive/trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [4]:
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['trade_volume-BUY-6-sec', 'moving_average_price-BUY-6-sec', 'weighted_moving_average_price-BUY-6-sec', 'exp_moving_average_price-BUY-6-sec', 'stochastic_oscillator-BUY-6-sec', 'trade_volume-BUY-12-sec', 'moving_average_price-BUY-12-sec', 'weighted_moving_average_price-BUY-12-sec', 'exp_moving_average_price-BUY-12-sec', 'stochastic_oscillator-BUY-12-sec', 'trade_volume-BUY-18-sec', 'moving_average_price-BUY-18-sec', 'weighted_moving_average_price-BUY-18-sec', 'exp_moving_average_price-BUY-18-sec', 'stochastic_oscillator-BUY-18-sec', 'trade_volume-BUY-24-sec', 'moving_average_price-BUY-24-sec', 'weighted_moving_average_price-BUY-24-sec', 'exp_moving_average_price-BUY-24-sec', 'stochastic_oscillator-BUY-24-sec', 'trade_volume-BUY-30-sec', 'moving_average_price-BUY-30-sec', 'weighted_moving_average_price-BUY-30-sec', 'exp_moving_average_price-BUY-30-sec', 'stochastic_oscillator-BUY-30-sec', 'trade_volume-BUY-36-sec', 'moving_average_price-BUY-36-sec', 'weighted_moving_average_price-BUY-36

In [5]:
X = df[x_columns]
secs = list(reversed(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns]))))))
delta_sec = secs[0] - secs[1]

for column in x_columns:
  name, side, sec, _ = column.split('-')
  if int(sec) == delta_sec:
    continue
  X[f'{name}-{side}-{sec}-sec'] -= X[f'{name}-{side}-{str(int(sec) - delta_sec)}-sec']

for column in x_columns:
  name, side, sec, _ = column.split('-')
  if 'price' in name and int(sec) == delta_sec:
    X = X.drop(f'{name}-{side}-{sec}-sec', axis=1)
X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,trade_volume-BUY-6-sec,stochastic_oscillator-BUY-6-sec,trade_volume-BUY-12-sec,moving_average_price-BUY-12-sec,weighted_moving_average_price-BUY-12-sec,exp_moving_average_price-BUY-12-sec,stochastic_oscillator-BUY-12-sec,trade_volume-BUY-18-sec,moving_average_price-BUY-18-sec,weighted_moving_average_price-BUY-18-sec,exp_moving_average_price-BUY-18-sec,stochastic_oscillator-BUY-18-sec,trade_volume-BUY-24-sec,moving_average_price-BUY-24-sec,weighted_moving_average_price-BUY-24-sec,exp_moving_average_price-BUY-24-sec,stochastic_oscillator-BUY-24-sec,trade_volume-BUY-30-sec,moving_average_price-BUY-30-sec,weighted_moving_average_price-BUY-30-sec,exp_moving_average_price-BUY-30-sec,stochastic_oscillator-BUY-30-sec,trade_volume-BUY-36-sec,moving_average_price-BUY-36-sec,weighted_moving_average_price-BUY-36-sec,exp_moving_average_price-BUY-36-sec,stochastic_oscillator-BUY-36-sec,trade_volume-BUY-42-sec,moving_average_price-BUY-42-sec,weighted_moving_average_price-BUY-42-sec,exp_moving_average_price-BUY-42-sec,stochastic_oscillator-BUY-42-sec,trade_volume-BUY-48-sec,moving_average_price-BUY-48-sec,weighted_moving_average_price-BUY-48-sec,exp_moving_average_price-BUY-48-sec,stochastic_oscillator-BUY-48-sec,trade_volume-BUY-54-sec,moving_average_price-BUY-54-sec,weighted_moving_average_price-BUY-54-sec,...,trade_volume-SELL-18-sec,moving_average_price-SELL-18-sec,weighted_moving_average_price-SELL-18-sec,exp_moving_average_price-SELL-18-sec,stochastic_oscillator-SELL-18-sec,trade_volume-SELL-24-sec,moving_average_price-SELL-24-sec,weighted_moving_average_price-SELL-24-sec,exp_moving_average_price-SELL-24-sec,stochastic_oscillator-SELL-24-sec,trade_volume-SELL-30-sec,moving_average_price-SELL-30-sec,weighted_moving_average_price-SELL-30-sec,exp_moving_average_price-SELL-30-sec,stochastic_oscillator-SELL-30-sec,trade_volume-SELL-36-sec,moving_average_price-SELL-36-sec,weighted_moving_average_price-SELL-36-sec,exp_moving_average_price-SELL-36-sec,stochastic_oscillator-SELL-36-sec,trade_volume-SELL-42-sec,moving_average_price-SELL-42-sec,weighted_moving_average_price-SELL-42-sec,exp_moving_average_price-SELL-42-sec,stochastic_oscillator-SELL-42-sec,trade_volume-SELL-48-sec,moving_average_price-SELL-48-sec,weighted_moving_average_price-SELL-48-sec,exp_moving_average_price-SELL-48-sec,stochastic_oscillator-SELL-48-sec,trade_volume-SELL-54-sec,moving_average_price-SELL-54-sec,weighted_moving_average_price-SELL-54-sec,exp_moving_average_price-SELL-54-sec,stochastic_oscillator-SELL-54-sec,trade_volume-SELL-60-sec,moving_average_price-SELL-60-sec,weighted_moving_average_price-SELL-60-sec,exp_moving_average_price-SELL-60-sec,stochastic_oscillator-SELL-60-sec
0,0.100,0.0,0.400,0.000000,0.000000,0.000000,0.000000,10.355,4291.266667,4291.113947,4291.580469,1.000000,0.500,-2.666667e-02,-1.049719e-03,0.000781,0.000000,10.655,4291.211282,4291.110911,4291.580469,1.000000,0.716,-1.112821e-01,-1.017205e-02,0.000757,0.000000,10.855,4291.111282,4291.095342,4291.580469,1.000000,0.916,-0.191282,-0.025212,0.000757,0.000000,10.855,4291.111282,4291.095342,...,0.400,4291.214286,4291.214286,4291.468750,1.000000,0.800,-1.506494e-01,-1.642857e-01,0.001562,0.000000,0.600,4291.189111,4291.192857,4291.468750,1.000000,3.500,-0.301611,-0.766028,0.001489,0.000000,0.800,4291.101611,4291.177656,4291.468750,1.000000,5.566,-0.401611,-0.867163,0.001489,0.000000,3.400,4291.049437,4291.116616,4291.468750,1.000000,15.547,-0.522770,-0.944837,0.001489,0.000000
1,0.200,0.0,0.300,0.000000,0.000000,0.000000,0.000000,0.400,4295.200000,4295.200000,4295.200000,0.000000,0.600,-9.094947e-13,1.818989e-12,0.000000,0.000000,0.650,4295.200000,4295.200000,4295.200000,0.000000,0.800,-9.094947e-13,1.818989e-12,0.000000,0.000000,1.399,4295.042105,4294.995634,4295.199963,0.600000,0.900,-0.022105,-0.017209,0.000000,0.000000,1.599,4295.108469,4295.057385,...,0.766,4295.100000,4295.100000,4295.100000,0.000000,4.084,-1.818989e-12,-1.818989e-12,0.000000,0.000000,0.766,4295.100000,4295.100000,4295.100000

In [6]:
commision = 0.0002
punch = commision * 5

y = pd.DataFrame(
    df[y_columns].apply(
        lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

       target
0           0
1           0
2           1
3           1
4           0
...       ...
38742       0
38743      -1
38744       0
38745       0
38746       0

[38747 rows x 1 columns]
target
 0        28863
-1         5479
 1         4405
dtype: int64


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [8]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=300,
        eval_metric=metrics.Accuracy(),
        random_seed=126,
        verbose=False,
        loss_function=metrics.MultiClass(),
    )
    
    cv_data = cv(
        Pool(X, y),
        model.get_params(),
        fold_count=3,
        shuffle=True,
        logging_level='Silent',
        stratified=True,
    )

    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy


params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=2,
    trials=trials,
    rstate=RandomState(126)
)

print(best)

100%|██████████| 2/2 [02:40<00:00, 80.00s/it, best loss: 0.24461260065842427]
{'l2_leaf_reg': 2.0, 'learning_rate': 0.11805599862470245}


In [9]:
params = {
    'iterations': 300,
    'l2_leaf_reg': best['l2_leaf_reg'],
    'learning_rate': best['learning_rate'],
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'loss_function': 'MultiClass',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
    verbose=10,
)

print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.7581


In [10]:
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[90:100])
print(predictions_probs[90:100])
print(y_test[90:100])

[[-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [ 0]
 [ 0]
 [ 0]]
[[0.43854381 0.19396752 0.36748867]
 [0.19602682 0.66465755 0.13931562]
 [0.19096911 0.62275276 0.18627813]
 [0.10012534 0.80721012 0.09266455]
 [0.18284245 0.6700532  0.14710435]
 [0.07725659 0.85880308 0.06394033]
 [0.355423   0.22877747 0.41579953]
 [0.08027186 0.86205696 0.05767117]
 [0.05452579 0.90513901 0.04033521]
 [0.22574516 0.50911687 0.26513797]]
       target
20233      -1
18782       0
26657       0
27456       0
36872       0
25891       0
33455      -1
23349       0
4176        0
37949       0


In [11]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


trade_volume-SELL-30-sec: 3.8061964611425347
moving_average_price-SELL-60-sec: 3.1520775323418295
trade_volume-BUY-60-sec: 3.009795843863408
trade_volume-SELL-54-sec: 2.528490158285116
trade_volume-BUY-42-sec: 2.4709841166250537
trade_volume-BUY-30-sec: 2.443549831235155
trade_volume-SELL-42-sec: 2.411292240754306
stochastic_oscillator-BUY-6-sec: 2.394864330495574
weighted_moving_average_price-SELL-60-sec: 2.2846272966166215
moving_average_price-SELL-12-sec: 2.2295172368030776
trade_volume-BUY-54-sec: 2.147277816274238
moving_average_price-BUY-18-sec: 2.0471848998542397
weighted_moving_average_price-SELL-12-sec: 2.039889424299367
trade_volume-SELL-36-sec: 1.977922160767204
weighted_moving_average_price-BUY-18-sec: 1.973957043757653
stochastic_oscillator-BUY-42-sec: 1.931933670258962
moving_average_price-SELL-48-sec: 1.931862554607304
trade_volume-SELL-12-sec: 1.762348926858085
trade_volume-BUY-12-sec: 1.7292232080177201
weighted_moving_average_price-BUY-54-sec: 1.723614286301477
moving

In [13]:
results = []

def f(prob_threshold, diff_threshold):
    p = np.array((list(map(lambda row:
                          1
                          if row[2] > prob_threshold
                              and row[2] == max(row)
                              and abs(row[2] - row[0]) > diff_threshold
                              and abs(row[2] - row[1]) > diff_threshold
                          else
                          -1
                          if row[0] > prob_threshold
                              and row[0] == max(row)
                              and abs(row[0] - row[2]) > diff_threshold
                              and abs(row[0] - row[1]) > diff_threshold
                          else 0,
                          predictions_probs))))
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
      t = int(y_test['target'][i:i+1])
      if p[i] == t != 0:
        result['correct_predictions'] += 1
      elif p[i] == t != 0:
        result['wrong_side'] += 1
      elif p[i] != 0 and t == 0:
        result['false_positive'] += 1
      elif p[i] == 0 and t != 0:
        result['false_negative'] += 1

    result['correct_predictions'] /= len(y_test)
    result['false_positive'] /= len(y_test)
    result['false_negative'] /= len(y_test)
    result['wrong_side'] /= len(y_test)
    result['prob_threshold'] = prob_threshold
    result['diff_threshold'] = diff_threshold
    results.append(result)

for prob_threshold in np.arange(0.3, 0.6, 0.01):
    for diff_threshold in np.arange(0.01, 0.02, 0.01):
      f(prob_threshold, diff_threshold)

results = list(reversed(sorted(results, key=lambda trade: trade['correct_predictions'])))

In [36]:
prob_threshold = results[0]['prob_threshold']
diff_threshold = results[0]['diff_threshold']
print(f"prob_threshold {prob_threshold}, diff_threshold {diff_threshold}")
p = np.array((list(map(lambda row:
                      1
                      if row[2] > prob_threshold
                          and row[2] == max(row)
                          and abs(row[2] - row[0]) > diff_threshold
                          and abs(row[2] - row[1]) > diff_threshold
                      else
                      -1
                      if row[0] > prob_threshold
                          and row[0] == max(row)
                          and abs(row[0] - row[2]) > diff_threshold
                          and abs(row[0] - row[1]) > diff_threshold
                      else 0,
                      predictions_probs))))
print('accuracy', accuracy_score(p, y_test))
result = {
    'correct_predictions': 0,
    'false_positive': 0,
    'false_negative': 0,
    'wrong_side': 0,
}
for i in range(len(p)):
  t = int(y_test['target'][i:i+1])
  if p[i] == t != 0:
    result['correct_predictions'] += 1
  elif p[i] == t != 0:
    result['wrong_side'] += 1
  elif p[i] != 0 and t == 0:
    result['false_positive'] += 1
  elif p[i] == 0 and t != 0:
    result['false_negative'] += 1

result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
result['false_positive_pc'] = result['false_positive'] / len(y_test)
result['false_negative_pc'] = result['false_negative'] / len(y_test)
result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
print(result)

prob_threshold 0.34, diff_threshold 0.01
accuracy 0.7576774193548387
{'correct_predictions': 133, 'false_positive': 58, 'false_negative': 1724, 'wrong_side': 0, 'correct_predictions_pc': 0.017161290322580645, 'false_positive_pc': 0.007483870967741935, 'false_negative_pc': 0.2224516129032258, 'wrong_side_pc': 0.0}


In [37]:
profit = (
      len(y) * result['correct_predictions_pc'] * (punch - 2 * commision)
      -
      len(y) * result['false_positive_pc'] * 2 * commision
      -
      len(y) * result['wrong_side_pc'] * (punch + 2 * commision)
)

print(
    'profit from 3000$ fot 2 months',
    profit * 3000,
    '$',
    profit * 100,
    '%'
)

profit from 3000$ fot 2 months 848.9342709677421 $ 28.29780903225807 %


In [38]:
correct_accuracy = result['correct_predictions'] / len(y_test[y_test['target'] != 0])
print(
f"Correct accuracy = correct prediction / total positive num = {correct_accuracy}"
)
FPR = result['false_positive'] / len(y_test[y_test['target'] == 0])
print(
f"FPR = false positive / total negative num = {FPR}"
)
FNR = result['false_negative'] / len(y_test[y_test['target'] != 0])
print(
f"FNR = false negative / total positive num = {FNR}"
)
cases_pc = (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) / len(y_test[y_test['target'] != 0]) * 100
probability_pc = result['correct_predictions'] / (result['correct_predictions'] + result['false_positive']) * 100
print(
f"Model in {cases_pc}% of cases with probability {probability_pc}% predict a market jump"
)

Correct accuracy = correct prediction / total positive num = 0.06810035842293907
FPR = false positive / total negative num = 0.010005175090564086
FNR = false negative / total positive num = 0.8827444956477215
Model in 9.779825908858166% of cases with probability 69.63350785340315% predict a market jump
